In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
from sklearn.preprocessing import MinMaxScaler
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
#Define function to add timesteps in datafram
def series_to_supervised(data, n_in=1, n_out=1, dropnan=True):
	"""
	Frame a time series as a supervised learning dataset.
	Arguments:
		data: Sequence of observations as a list or NumPy array.
		n_in: Number of lag observations as input (X).
		n_out: Number of observations as output (y).
		dropnan: Boolean whether or not to drop rows with NaN values.
	Returns:
		Pandas DataFrame of series framed for supervised learning.
	"""
	n_vars = 1 if type(data) is list else data.shape[1]
	df = pd.DataFrame(data)
	cols, names = list(), list()
	# input sequence (t-n, ... t-1)
	for i in range(n_in, 0, -1):
		cols.append(df.shift(i))
		names += [('var%d(t-%d)' % (j+1, i)) for j in range(n_vars)]
	# forecast sequence (t, t+1, ... t+n)
	for i in range(0, n_out):
		cols.append(df.shift(-i))
		if i == 0:
			names += [('var%d(t)' % (j+1)) for j in range(n_vars)]
		else:
			names += [('var%d(t+%d)' % (j+1, i)) for j in range(n_vars)]
	# put it all together
	agg = pd.concat(cols, axis=1)
	agg.columns = names
	# drop rows with NaN values
	if dropnan:
		agg.dropna(inplace=True)
	return agg

In [3]:
#Add timesteps to the dataframe
dfm = pd.read_csv('mergeddata.csv', header=0, index_col=1)
values = dfm.values
n_days = 20
dfm = dfm.drop(['number'], axis = 1)
dfm.head()

,Price,High,Low
Date,,,
"Nov 21, 2018",1.1371,1.1410,1.1366
"Nov 20, 2018",1.1372,1.1472,1.1359
"Nov 19, 2018",1.1453,1.1467,1.1393
"Nov 16, 2018",1.1420,1.1423,1.1320
"Nov 15, 2018",1.1328,1.1364,1.1271


In [4]:
dfm = dfm.reindex(index=dfm.index[::-1])
dfm.head()

,Price,High,Low
Date,,,
"Dec 27, 1979",1.5074,1.5074,1.5074
"Dec 28, 1979",1.5076,1.5076,1.5076
"Dec 31, 1979",1.5081,1.5081,1.5081
"Jan 02, 1980",1.5149,1.5149,1.5149
"Jan 03, 1980",1.5177,1.5177,1.5177


In [5]:
dfm_series = series_to_supervised(dfm, n_days, 1)
dfm_series.head()

,var1(t-20),var2(t-20),var3(t-20),var1(t-19),var2(t-19),var3(t-19),var1(t-18),var2(t-18),var3(t-18),var1(t-17),...,var3(t-3),var1(t-2),var2(t-2),var3(t-2),var1(t-1),var2(t-1),var3(t-1),var1(t),var2(t),var3(t)
Date,,,,,,,,,,,,,,,,,,,,,
"Jan 25, 1980",1.5074,1.5074,1.5074,1.5076,1.5076,1.5076,1.5081,1.5081,1.5081,1.5149,...,1.5006,1.5035,1.5035,1.5035,1.5026,1.5026,1.5026,1.5004,1.5004,1.5004
"Jan 28, 1980",1.5076,1.5076,1.5076,1.5081,1.5081,1.5081,1.5149,1.5149,1.5149,1.5177,...,1.5035,1.5026,1.5026,1.5026,1.5004,1.5004,1.5004,1.4993,1.4993,1.4993
"Jan 29, 1980",1.5081,1.5081,1.5081,1.5149,1.5149,1.5149,1.5177,1.5177,1.5177,1.5129,...,1.5026,1.5004,1.5004,1.5004,1.4993,1.4993,1.4993,1.4979,1.4979,1.4979
"Jan 30, 1980",1.5149,1.5149,1.5149,1.5177,1.5177,1.5177,1.5129,1.5129,1.5129,1.5168,...,1.5004,1.4993,1.4993,1.4993,1.4979,1.4979,1.4979,1.4986,1.4986,1.4986
"Jan 31, 1980",1.5177,1.5177,1.5177,1.5129,1.5129,1.5129,1.5168,1.5168,1.5168,1.5108,...,1.4993,1.4979,1.4979,1.4979,1.4986,1.4986,1.4986,1.4945,1.4945,1.4945


In [6]:
#get var1(t) to the end
swapcol = list(dfm_series)
#print(swapcol)
swapcol[-3], swapcol[-1] = swapcol[-1], swapcol[-3]
dfm_series = dfm_series.loc[:,swapcol]

In [7]:
#dfm_series.index.name = 'Date'
dfm_series.to_csv('mergeddataseries.csv')
dfm_series.head()

,var1(t-20),var2(t-20),var3(t-20),var1(t-19),var2(t-19),var3(t-19),var1(t-18),var2(t-18),var3(t-18),var1(t-17),...,var3(t-3),var1(t-2),var2(t-2),var3(t-2),var1(t-1),var2(t-1),var3(t-1),var3(t),var2(t),var1(t)
Date,,,,,,,,,,,,,,,,,,,,,
"Jan 25, 1980",1.5074,1.5074,1.5074,1.5076,1.5076,1.5076,1.5081,1.5081,1.5081,1.5149,...,1.5006,1.5035,1.5035,1.5035,1.5026,1.5026,1.5026,1.5004,1.5004,1.5004
"Jan 28, 1980",1.5076,1.5076,1.5076,1.5081,1.5081,1.5081,1.5149,1.5149,1.5149,1.5177,...,1.5035,1.5026,1.5026,1.5026,1.5004,1.5004,1.5004,1.4993,1.4993,1.4993
"Jan 29, 1980",1.5081,1.5081,1.5081,1.5149,1.5149,1.5149,1.5177,1.5177,1.5177,1.5129,...,1.5026,1.5004,1.5004,1.5004,1.4993,1.4993,1.4993,1.4979,1.4979,1.4979
"Jan 30, 1980",1.5149,1.5149,1.5149,1.5177,1.5177,1.5177,1.5129,1.5129,1.5129,1.5168,...,1.5004,1.4993,1.4993,1.4993,1.4979,1.4979,1.4979,1.4986,1.4986,1.4986
"Jan 31, 1980",1.5177,1.5177,1.5177,1.5129,1.5129,1.5129,1.5168,1.5168,1.5168,1.5108,...,1.4993,1.4979,1.4979,1.4979,1.4986,1.4986,1.4986,1.4945,1.4945,1.4945
